In [2]:
import numpy as np
from sqlalchemy import create_engine
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from collections import OrderedDict
import imblearn.over_sampling
from sklearn.metrics import roc_auc_score
import xgboost as xgb

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
cnx = create_engine('postgresql://postgres@localhost:5432/instacart')

/home/ubuntu/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
df_orders = pd.read_sql_query('''SELECT * FROM orders_2''',cnx)
df_order_products_prior = pd.read_sql_query('''SELECT * FROM order_products_prior''',cnx)
df_order_products_train = pd.read_sql_query('''SELECT * FROM order_products_train''',cnx)
df_products = pd.read_sql_query('''SELECT * FROM products_full''',cnx)
df_users = pd.read_sql_query('''SELECT * FROM users''',cnx)

In [3]:
# with open('orders.pkl', 'rb') as picklefile:
#     df_orders = pickle.load(picklefile)
with open('order_products_prior2.pkl', 'rb') as picklefile:
    df_order_products_prior = pickle.load(picklefile)
# with open('order_products_train.pkl', 'rb') as picklefile:
#     df_order_products_train = pickle.load(picklefile)
# with open('products.pkl', 'rb') as picklefile:
#     df_products = pickle.load(picklefile)
# with open('users.pkl', 'rb') as picklefile:
#     df_users = pickle.load(picklefile)

In [8]:
# Combining all tables together

df_order_products_train = df_order_products_train.merge(df_orders.drop('eval_set', axis=1), on='order_id')
df_order_products_train = df_order_products_train.merge(df_products, on='product_id')
# df_order_products_prior = df_order_products_prior.merge(df_orders.drop('eval_set', axis=1), on='order_id')
# df_order_products_prior = df_order_products_prior.merge(df_products, on='product_id')

In [9]:
# Pickling these files for re-import when needed


# with open('orders.pkl', 'wb') as picklefile:
#     pickle.dump(df_orders, picklefile)
# with open('order_products_prior2.pkl', 'wb') as picklefile:
#     pickle.dump(df_order_products_prior, picklefile)
with open('order_products_train2.pkl', 'wb') as picklefile:
    pickle.dump(df_order_products_train, picklefile)
# with open('products.pkl', 'wb') as picklefile:
#     pickle.dump(df_products, picklefile)
# with open('users.pkl', 'wb') as picklefile:
#     pickle.dump(df_users, picklefile)

In [7]:
df_user_product = (df_order_products_prior.groupby(['product_id','user_id'],as_index=False) 
                                          .agg({'order_id':'count'}) 
                                          .rename(columns={'order_id':'user_product_total_orders'}))

train_ids = df_order_products_train['user_id'].unique() 
df_X = df_user_product[df_user_product['user_id'].isin(train_ids)]

In [8]:
train_carts = (df_order_products_train.groupby('user_id',as_index=False)
                                      .agg({'product_id':(lambda x: set(x))})
                                      .rename(columns={'product_id':'latest_cart'}))

df_X = df_X.merge(train_carts, on='user_id')
df_X['in_cart'] = (df_X.apply(lambda row: row['product_id'] in row['latest_cart'], axis=1).astype(int))

# Initial Product Features

In [9]:
prod_features = ['product_total_orders','product_avg_add_to_cart_order','product_avg_order_number','product_avg_dow',\
                'product_avg_hour_of_day','product_total_times_reordered','product_reordered_rate']

df_prod_features = (df_order_products_prior.groupby(['product_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique'),
                                                    ('add_to_cart_order','mean'),
                                                    ('order_number', 'mean'),
                                                    ('order_dow', 'mean'),
                                                    ('order_hour_of_day', 'mean'),
                                                    ('reordered', ['sum','mean'])])))
df_prod_features.columns = ['product_id'] + prod_features

df_X = df_X.merge(df_prod_features, on='product_id')

# Initial User Features

In [10]:
user_features = ['user_total_orders','user_avg_cartsize','user_total_products','user_avg_days_since_prior_order',\
                'user_avg_dow','user_avg_hour_of_day','user_total_reordered','user_reordered_percentage']

df_user_features = (df_order_products_prior.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id',['nunique', (lambda x: x.shape[0] / x.nunique())]),
                                                    ('product_id','nunique'),
                                                    ('days_since_prior_order','mean'),
                                                    ('order_dow', 'mean'),
                                                    ('order_hour_of_day', 'mean'),
                                                    ('reordered', ['sum','mean'])])))

df_user_features.columns = ['user_id'] + user_features
df_user_features['user_avg_reordered'] = df_user_features['user_avg_cartsize'] * df_user_features['user_reordered_percentage']

df_X = df_X.merge(df_user_features, on='user_id')

# Initial User-Product Features

In [11]:
user_prod_features = ['user_product_avg_add_to_cart_order','user_product_avg_order_dow',\
                      'user_product_avg_order_hour_of_day','user_product_avg_days_since_prior_order']

df_user_prod_features = (df_order_products_prior.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict(
                                                     [('add_to_cart_order','mean'),
                                                      ('order_dow', 'mean'),
                                                      ('order_hour_of_day', 'mean'),
                                                      ('days_since_prior_order', 'mean')])))

df_user_prod_features.columns = ['product_id','user_id'] + user_prod_features 

df_X = df_X.merge(df_user_prod_features,on=['user_id','product_id'])

df_X['user_product_order_freq'] = df_X['user_product_total_orders'] / df_X['user_total_orders'] 

In [12]:
# Replace NA values in the user_product_avg_days_since_prior_order column with -1 to indicate it was first purchase of item

df_X = df_X.fillna(-1)

In [13]:
# Save file out to csv
df_X.to_csv('instacart_df_X.csv', index=False)

# Adding additional user-product features

In [21]:
df_X = pd.read_csv('instacart_df_X.csv')

In [7]:
df_users1 = df_users[:20000]
df_users2 = df_users[20000:40000]
df_users3 = df_users[40000:60000]
df_users4 = df_users[60000:80000]
df_users5 = df_users[80000:100000]
df_users6 = df_users[100000:120000]
df_users7 = df_users[120000:]

In [19]:
df_orders_since_reorder = df_order_products_prior[['product_id','user_id','order_number']]

In [20]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder1 = df_orders_since_reorder.merge(df_users1, on=['user_id'])

df_orders_since_reorder1 = (df_orders_since_reorder1.merge(df_orders_since_reorder1, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder1 = df_orders_since_reorder1[df_orders_since_reorder1['order_number_x'] > \
                                                  df_orders_since_reorder1['order_number_y']]

df_orders_since_reorder1 = df_orders_since_reorder1.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder1['orders_since_prior'] = df_orders_since_reorder1['order_number_x'] - \
                                                df_orders_since_reorder1['order_number_y']
    
df_orders_since_reorder1 = df_orders_since_reorder1.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder1 = df_orders_since_reorder1.drop(columns='order_number_y')

In [21]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder2 = df_orders_since_reorder.merge(df_users2, on=['user_id'])

df_orders_since_reorder2 = (df_orders_since_reorder2.merge(df_orders_since_reorder2, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder2 = df_orders_since_reorder2[df_orders_since_reorder2['order_number_x'] > \
                                                  df_orders_since_reorder2['order_number_y']]

df_orders_since_reorder2 = df_orders_since_reorder2.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder2['orders_since_prior'] = df_orders_since_reorder2['order_number_x'] - \
                                                df_orders_since_reorder2['order_number_y']
    
df_orders_since_reorder2 = df_orders_since_reorder2.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder2 = df_orders_since_reorder2.drop(columns='order_number_y')

In [22]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder3 = df_orders_since_reorder.merge(df_users3, on=['user_id'])

df_orders_since_reorder3 = (df_orders_since_reorder3.merge(df_orders_since_reorder3, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder3 = df_orders_since_reorder3[df_orders_since_reorder3['order_number_x'] > \
                                                  df_orders_since_reorder3['order_number_y']]

df_orders_since_reorder3 = df_orders_since_reorder3.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder3['orders_since_prior'] = df_orders_since_reorder3['order_number_x'] - \
                                                df_orders_since_reorder3['order_number_y']
    
df_orders_since_reorder3 = df_orders_since_reorder3.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder3 = df_orders_since_reorder3.drop(columns='order_number_y')

In [23]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder4 = df_orders_since_reorder.merge(df_users4, on=['user_id'])

df_orders_since_reorder4 = (df_orders_since_reorder4.merge(df_orders_since_reorder4, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder4 = df_orders_since_reorder4[df_orders_since_reorder4['order_number_x'] > \
                                                  df_orders_since_reorder4['order_number_y']]

df_orders_since_reorder4 = df_orders_since_reorder4.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder4['orders_since_prior'] = df_orders_since_reorder4['order_number_x'] - \
                                                df_orders_since_reorder4['order_number_y']
    
df_orders_since_reorder4 = df_orders_since_reorder4.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder4 = df_orders_since_reorder4.drop(columns='order_number_y')

In [24]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder5 = df_orders_since_reorder.merge(df_users5, on=['user_id'])

df_orders_since_reorder5 = (df_orders_since_reorder5.merge(df_orders_since_reorder5, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder5 = df_orders_since_reorder5[df_orders_since_reorder5['order_number_x'] > \
                                                  df_orders_since_reorder5['order_number_y']]

df_orders_since_reorder5 = df_orders_since_reorder5.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder5['orders_since_prior'] = df_orders_since_reorder5['order_number_x'] - \
                                                df_orders_since_reorder5['order_number_y']
    
df_orders_since_reorder5 = df_orders_since_reorder5.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder5 = df_orders_since_reorder5.drop(columns='order_number_y')

In [25]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder6 = df_orders_since_reorder.merge(df_users6, on=['user_id'])

df_orders_since_reorder6 = (df_orders_since_reorder6.merge(df_orders_since_reorder6, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder6 = df_orders_since_reorder6[df_orders_since_reorder6['order_number_x'] > \
                                                  df_orders_since_reorder6['order_number_y']]

df_orders_since_reorder6 = df_orders_since_reorder6.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder6['orders_since_prior'] = df_orders_since_reorder6['order_number_x'] - \
                                                df_orders_since_reorder6['order_number_y']
    
df_orders_since_reorder6 = df_orders_since_reorder6.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder6 = df_orders_since_reorder6.drop(columns='order_number_y')

In [26]:
# Adding feature for number of orders since prior time product was ordered
df_orders_since_reorder7 = df_orders_since_reorder.merge(df_users7, on=['user_id'])

df_orders_since_reorder7 = (df_orders_since_reorder7.merge(df_orders_since_reorder7, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder7 = df_orders_since_reorder7[df_orders_since_reorder7['order_number_x'] > \
                                                  df_orders_since_reorder7['order_number_y']]

df_orders_since_reorder7 = df_orders_since_reorder7.groupby(['product_id', 'user_id', 'order_number_x'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder7['orders_since_prior'] = df_orders_since_reorder7['order_number_x'] - \
                                                df_orders_since_reorder7['order_number_y']
    
df_orders_since_reorder7 = df_orders_since_reorder7.rename(columns={'order_number_x':'order_number'})
df_orders_since_reorder7 = df_orders_since_reorder7.drop(columns='order_number_y')

In [27]:
df_orders_since_reorder_list = [df_orders_since_reorder1, df_orders_since_reorder2, df_orders_since_reorder3, \
                                df_orders_since_reorder4, df_orders_since_reorder5, df_orders_since_reorder6, \
                               df_orders_since_reorder7]

In [28]:
df_orders_since_reorder = pd.concat(df_orders_since_reorder_list)

In [29]:
df_order_products_prior = df_order_products_prior.merge(df_orders_since_reorder, on=['product_id','user_id','order_number'], how='left')

In [30]:
user_prod_features2 = ['user_product_orders_since_prior']

df_user_prod_features2 = (df_order_products_prior.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict(
                                                     [('orders_since_prior','mean')])))

df_user_prod_features2.columns = ['product_id','user_id'] + user_prod_features2

In [31]:
df_X = df_X.merge(df_user_prod_features2,on=['user_id','product_id'])

df_X = df_X.fillna(-1)

In [32]:
del df_orders_since_reorder
del df_orders_since_reorder_list
del df_orders_since_reorder1
del df_orders_since_reorder2
del df_orders_since_reorder3
del df_orders_since_reorder4
del df_orders_since_reorder5
del df_orders_since_reorder6
del df_orders_since_reorder7

In [33]:
# Save file out to csv
df_X.to_csv('instacart_df_X.csv', index=False)

In [3]:
df_X = pd.read_csv('instacart_df_X.csv')

In [6]:
df_users1 = df_users[:10000]
df_users2 = df_users[10000:20000]
df_users3 = df_users[20000:30000]
df_users4 = df_users[30000:40000]
df_users5 = df_users[40000:50000]
df_users6 = df_users[50000:60000]
df_users7 = df_users[60000:70000]
df_users8 = df_users[70000:80000]
df_users9 = df_users[80000:90000]
df_users10 = df_users[90000:100000]
df_users11 = df_users[100000:110000]
df_users12 = df_users[110000:120000]
df_users13 = df_users[120000:130000]
df_users14 = df_users[130000:]

In [5]:
df_orders_since_reorder_dpt = df_order_products_prior[['product_id','user_id','order_number','department']]

In [7]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt1 = df_orders_since_reorder_dpt.merge(df_users1, on=['user_id'])

df_orders_since_reorder_dpt1 = (df_orders_since_reorder_dpt1.merge(df_orders_since_reorder_dpt1, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt1 = df_orders_since_reorder_dpt1[df_orders_since_reorder_dpt1['order_number_x'] > \
                                                  df_orders_since_reorder_dpt1['order_number_y']]

df_orders_since_reorder_dpt1 = df_orders_since_reorder_dpt1.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt1['orders_since_prior_department'] = df_orders_since_reorder_dpt1['order_number_x'] - \
                                                df_orders_since_reorder_dpt1['order_number_y']
    
df_orders_since_reorder_dpt1 = df_orders_since_reorder_dpt1.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt1 = df_orders_since_reorder_dpt1.drop(columns='order_number_y')

In [8]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt2 = df_orders_since_reorder_dpt.merge(df_users2, on=['user_id'])

df_orders_since_reorder_dpt2 = (df_orders_since_reorder_dpt2.merge(df_orders_since_reorder_dpt2, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt2 = df_orders_since_reorder_dpt2[df_orders_since_reorder_dpt2['order_number_x'] > \
                                                  df_orders_since_reorder_dpt2['order_number_y']]

df_orders_since_reorder_dpt2 = df_orders_since_reorder_dpt2.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt2['orders_since_prior_department'] = df_orders_since_reorder_dpt2['order_number_x'] - \
                                                df_orders_since_reorder_dpt2['order_number_y']
    
df_orders_since_reorder_dpt2 = df_orders_since_reorder_dpt2.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt2 = df_orders_since_reorder_dpt2.drop(columns='order_number_y')

In [9]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt3 = df_orders_since_reorder_dpt.merge(df_users3, on=['user_id'])

df_orders_since_reorder_dpt3 = (df_orders_since_reorder_dpt3.merge(df_orders_since_reorder_dpt3, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt3 = df_orders_since_reorder_dpt3[df_orders_since_reorder_dpt3['order_number_x'] > \
                                                  df_orders_since_reorder_dpt3['order_number_y']]

df_orders_since_reorder_dpt3 = df_orders_since_reorder_dpt3.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt3['orders_since_prior_department'] = df_orders_since_reorder_dpt3['order_number_x'] - \
                                                df_orders_since_reorder_dpt3['order_number_y']
    
df_orders_since_reorder_dpt3 = df_orders_since_reorder_dpt3.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt3 = df_orders_since_reorder_dpt3.drop(columns='order_number_y')

In [10]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt4 = df_orders_since_reorder_dpt.merge(df_users4, on=['user_id'])

df_orders_since_reorder_dpt4 = (df_orders_since_reorder_dpt4.merge(df_orders_since_reorder_dpt4, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt4 = df_orders_since_reorder_dpt4[df_orders_since_reorder_dpt4['order_number_x'] > \
                                                  df_orders_since_reorder_dpt4['order_number_y']]

df_orders_since_reorder_dpt4 = df_orders_since_reorder_dpt4.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt4['orders_since_prior_department'] = df_orders_since_reorder_dpt4['order_number_x'] - \
                                                df_orders_since_reorder_dpt4['order_number_y']
    
df_orders_since_reorder_dpt4 = df_orders_since_reorder_dpt4.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt4 = df_orders_since_reorder_dpt4.drop(columns='order_number_y')

In [11]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt5 = df_orders_since_reorder_dpt.merge(df_users5, on=['user_id'])

df_orders_since_reorder_dpt5 = (df_orders_since_reorder_dpt5.merge(df_orders_since_reorder_dpt5, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt5 = df_orders_since_reorder_dpt5[df_orders_since_reorder_dpt5['order_number_x'] > \
                                                  df_orders_since_reorder_dpt5['order_number_y']]

df_orders_since_reorder_dpt5 = df_orders_since_reorder_dpt5.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt5['orders_since_prior_department'] = df_orders_since_reorder_dpt5['order_number_x'] - \
                                                df_orders_since_reorder_dpt5['order_number_y']
    
df_orders_since_reorder_dpt5 = df_orders_since_reorder_dpt5.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt5 = df_orders_since_reorder_dpt5.drop(columns='order_number_y')

In [12]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt6 = df_orders_since_reorder_dpt.merge(df_users6, on=['user_id'])

df_orders_since_reorder_dpt6 = (df_orders_since_reorder_dpt6.merge(df_orders_since_reorder_dpt6, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt6 = df_orders_since_reorder_dpt6[df_orders_since_reorder_dpt6['order_number_x'] > \
                                                  df_orders_since_reorder_dpt6['order_number_y']]

df_orders_since_reorder_dpt6 = df_orders_since_reorder_dpt6.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt6['orders_since_prior_department'] = df_orders_since_reorder_dpt6['order_number_x'] - \
                                                df_orders_since_reorder_dpt6['order_number_y']
    
df_orders_since_reorder_dpt6 = df_orders_since_reorder_dpt6.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt6 = df_orders_since_reorder_dpt6.drop(columns='order_number_y')

In [13]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt7 = df_orders_since_reorder_dpt.merge(df_users7, on=['user_id'])

df_orders_since_reorder_dpt7 = (df_orders_since_reorder_dpt7.merge(df_orders_since_reorder_dpt7, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt7 = df_orders_since_reorder_dpt7[df_orders_since_reorder_dpt7['order_number_x'] > \
                                                  df_orders_since_reorder_dpt7['order_number_y']]

df_orders_since_reorder_dpt7 = df_orders_since_reorder_dpt7.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt7['orders_since_prior_department'] = df_orders_since_reorder_dpt7['order_number_x'] - \
                                                df_orders_since_reorder_dpt7['order_number_y']
    
df_orders_since_reorder_dpt7 = df_orders_since_reorder_dpt7.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt7 = df_orders_since_reorder_dpt7.drop(columns='order_number_y')

In [14]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt8 = df_orders_since_reorder_dpt.merge(df_users8, on=['user_id'])

df_orders_since_reorder_dpt8 = (df_orders_since_reorder_dpt8.merge(df_orders_since_reorder_dpt8, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt8 = df_orders_since_reorder_dpt8[df_orders_since_reorder_dpt8['order_number_x'] > \
                                                  df_orders_since_reorder_dpt8['order_number_y']]

df_orders_since_reorder_dpt8 = df_orders_since_reorder_dpt8.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt8['orders_since_prior_department'] = df_orders_since_reorder_dpt8['order_number_x'] - \
                                                df_orders_since_reorder_dpt8['order_number_y']
    
df_orders_since_reorder_dpt8 = df_orders_since_reorder_dpt8.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt8 = df_orders_since_reorder_dpt8.drop(columns='order_number_y')

In [15]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt9 = df_orders_since_reorder_dpt.merge(df_users9, on=['user_id'])

df_orders_since_reorder_dpt9 = (df_orders_since_reorder_dpt9.merge(df_orders_since_reorder_dpt9, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt9 = df_orders_since_reorder_dpt9[df_orders_since_reorder_dpt9['order_number_x'] > \
                                                  df_orders_since_reorder_dpt9['order_number_y']]

df_orders_since_reorder_dpt9 = df_orders_since_reorder_dpt9.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt9['orders_since_prior_department'] = df_orders_since_reorder_dpt9['order_number_x'] - \
                                                df_orders_since_reorder_dpt9['order_number_y']
    
df_orders_since_reorder_dpt9 = df_orders_since_reorder_dpt9.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt9 = df_orders_since_reorder_dpt9.drop(columns='order_number_y')

In [16]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt10 = df_orders_since_reorder_dpt.merge(df_users10, on=['user_id'])

df_orders_since_reorder_dpt10 = (df_orders_since_reorder_dpt10.merge(df_orders_since_reorder_dpt10, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt10 = df_orders_since_reorder_dpt10[df_orders_since_reorder_dpt10['order_number_x'] > \
                                                  df_orders_since_reorder_dpt10['order_number_y']]

df_orders_since_reorder_dpt10 = df_orders_since_reorder_dpt10.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt10['orders_since_prior_department'] = df_orders_since_reorder_dpt10['order_number_x'] - \
                                                df_orders_since_reorder_dpt10['order_number_y']
    
df_orders_since_reorder_dpt10 = df_orders_since_reorder_dpt10.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt10 = df_orders_since_reorder_dpt10.drop(columns='order_number_y')

In [17]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt11 = df_orders_since_reorder_dpt.merge(df_users11, on=['user_id'])

df_orders_since_reorder_dpt11 = (df_orders_since_reorder_dpt11.merge(df_orders_since_reorder_dpt11, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt11 = df_orders_since_reorder_dpt11[df_orders_since_reorder_dpt11['order_number_x'] > \
                                                  df_orders_since_reorder_dpt11['order_number_y']]

df_orders_since_reorder_dpt11 = df_orders_since_reorder_dpt11.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt11['orders_since_prior_department'] = df_orders_since_reorder_dpt11['order_number_x'] - \
                                                df_orders_since_reorder_dpt11['order_number_y']
    
df_orders_since_reorder_dpt11 = df_orders_since_reorder_dpt11.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt11 = df_orders_since_reorder_dpt11.drop(columns='order_number_y')

In [18]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt12 = df_orders_since_reorder_dpt.merge(df_users12, on=['user_id'])

df_orders_since_reorder_dpt12 = (df_orders_since_reorder_dpt12.merge(df_orders_since_reorder_dpt12, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt12 = df_orders_since_reorder_dpt12[df_orders_since_reorder_dpt12['order_number_x'] > \
                                                  df_orders_since_reorder_dpt12['order_number_y']]

df_orders_since_reorder_dpt12 = df_orders_since_reorder_dpt12.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt12['orders_since_prior_department'] = df_orders_since_reorder_dpt12['order_number_x'] - \
                                                df_orders_since_reorder_dpt12['order_number_y']
    
df_orders_since_reorder_dpt12 = df_orders_since_reorder_dpt12.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt12 = df_orders_since_reorder_dpt12.drop(columns='order_number_y')

In [19]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt13 = df_orders_since_reorder_dpt.merge(df_users13, on=['user_id'])

df_orders_since_reorder_dpt13 = (df_orders_since_reorder_dpt13.merge(df_orders_since_reorder_dpt13, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt13 = df_orders_since_reorder_dpt13[df_orders_since_reorder_dpt13['order_number_x'] > \
                                                  df_orders_since_reorder_dpt13['order_number_y']]

df_orders_since_reorder_dpt13 = df_orders_since_reorder_dpt13.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt13['orders_since_prior_department'] = df_orders_since_reorder_dpt13['order_number_x'] - \
                                                df_orders_since_reorder_dpt13['order_number_y']
    
df_orders_since_reorder_dpt13 = df_orders_since_reorder_dpt13.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt13 = df_orders_since_reorder_dpt13.drop(columns='order_number_y')

In [20]:
# Adding feature for number of orders since user ordered product from same department
df_orders_since_reorder_dpt14 = df_orders_since_reorder_dpt.merge(df_users14, on=['user_id'])

df_orders_since_reorder_dpt14 = (df_orders_since_reorder_dpt14.merge(df_orders_since_reorder_dpt14, on=['user_id','department'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_orders_since_reorder_dpt14 = df_orders_since_reorder_dpt14[df_orders_since_reorder_dpt14['order_number_x'] > \
                                                  df_orders_since_reorder_dpt14['order_number_y']]

df_orders_since_reorder_dpt14 = df_orders_since_reorder_dpt14.groupby(['product_id_x','user_id','order_number_x','department'])\
                                                        ['order_number_y'].max().reset_index()

df_orders_since_reorder_dpt14['orders_since_prior_department'] = df_orders_since_reorder_dpt14['order_number_x'] - \
                                                df_orders_since_reorder_dpt14['order_number_y']
    
df_orders_since_reorder_dpt14 = df_orders_since_reorder_dpt14.rename(columns={'order_number_x':'order_number',\
                                                                             'product_id_x':'product_id'})
df_orders_since_reorder_dpt14 = df_orders_since_reorder_dpt14.drop(columns='order_number_y')

In [25]:
df_orders_since_reorder_dpt_list = [df_orders_since_reorder_dpt1, df_orders_since_reorder_dpt2, df_orders_since_reorder_dpt3, \
                                df_orders_since_reorder_dpt4, df_orders_since_reorder_dpt5, df_orders_since_reorder_dpt6, \
                                df_orders_since_reorder_dpt7, df_orders_since_reorder_dpt8, df_orders_since_reorder_dpt9, \
                                df_orders_since_reorder_dpt10, df_orders_since_reorder_dpt11, df_orders_since_reorder_dpt12, \
                                df_orders_since_reorder_dpt13, df_orders_since_reorder_dpt14]

In [26]:
df_orders_since_reorder_dpt = pd.concat(df_orders_since_reorder_dpt_list)

In [27]:
df_order_products_prior = df_order_products_prior.merge(df_orders_since_reorder_dpt, on=['product_id','user_id','order_number'], how='left')

In [28]:
user_prod_features2 = ['user_product_orders_since_prior_department']

df_user_prod_features2 = (df_order_products_prior.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict(
                                                     [('orders_since_prior_department','mean')])))

df_user_prod_features2.columns = ['product_id','user_id'] + user_prod_features2

In [29]:
df_X = df_X.merge(df_user_prod_features2,on=['user_id','product_id'])

df_X = df_X.fillna(-1)

In [30]:
# Save file out to csv
df_X.to_csv('instacart_df_X.csv', index=False)

In [ ]:
del df_orders_since_reorder_dpt
del df_orders_since_reorder_dpt_list
del df_orders_since_reorder_dpt1
del df_orders_since_reorder_dpt2
del df_orders_since_reorder_dpt3
del df_orders_since_reorder_dpt4
del df_orders_since_reorder_dpt5
del df_orders_since_reorder_dpt6
del df_orders_since_reorder_dpt7
del df_orders_since_reorder_dpt8
del df_orders_since_reorder_dpt9
del df_orders_since_reorder_dpt10
del df_orders_since_reorder_dpt11
del df_orders_since_reorder_dpt12
del df_orders_since_reorder_dpt13
del df_orders_since_reorder_dpt14

# Adding more product features

In [3]:
df_X = pd.read_csv('instacart_df_X.csv')

In [10]:
user_current_order = df_order_products_train.groupby(['user_id', 'order_dow', 'order_hour_of_day'])['order_id'].nunique().reset_index()
user_current_order = user_current_order.drop(columns='order_id')
df_X = df_X.merge(user_current_order, on='user_id')

In [11]:
df_X['product_avg_dow_delta'] = df_X['order_dow'] - df_X['product_avg_dow']
df_X['product_avg_hod_delta'] = df_X['order_hour_of_day'] - df_X['product_avg_hour_of_day']

In [12]:
df_X = df_X.drop(columns=['order_dow', 'order_hour_of_day'])

# Adding additional User-product features

In [13]:
df_users1 = df_users[:20000]
df_users2 = df_users[20000:40000]
df_users3 = df_users[40000:60000]
df_users4 = df_users[60000:80000]
df_users5 = df_users[80000:100000]
df_users6 = df_users[100000:120000]
df_users7 = df_users[120000:]

In [15]:
df_consecutive_orders = df_order_products_prior[['product_id','user_id','order_number']]

In [16]:
# Adding feature for consecutive orders
df_consecutive_orders1 = df_consecutive_orders.merge(df_users1, on=['user_id'])

df_consecutive_orders1 = (df_consecutive_orders1.merge(df_consecutive_orders1, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders1 = df_consecutive_orders1[df_consecutive_orders1['order_number_x'] - \
                                                  df_consecutive_orders1['order_number_y'] == 1]

df_consecutive_orders1['ones'] = 1

df_consecutive_orders1 = df_consecutive_orders1.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders1 = df_consecutive_orders1.rename(columns={'ones':'consecutive_reorders'})

In [17]:
# Adding feature for consecutive orders
df_consecutive_orders2 = df_consecutive_orders.merge(df_users2, on=['user_id'])

df_consecutive_orders2 = (df_consecutive_orders2.merge(df_consecutive_orders2, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders2 = df_consecutive_orders2[df_consecutive_orders2['order_number_x'] - \
                                                  df_consecutive_orders2['order_number_y'] == 1]

df_consecutive_orders2['ones'] = 1

df_consecutive_orders2 = df_consecutive_orders2.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders2 = df_consecutive_orders2.rename(columns={'ones':'consecutive_reorders'})

In [18]:
# Adding feature for consecutive orders
df_consecutive_orders3 = df_consecutive_orders.merge(df_users3, on=['user_id'])

df_consecutive_orders3 = (df_consecutive_orders3.merge(df_consecutive_orders3, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders3 = df_consecutive_orders3[df_consecutive_orders3['order_number_x'] - \
                                                  df_consecutive_orders3['order_number_y'] == 1]

df_consecutive_orders3['ones'] = 1

df_consecutive_orders3 = df_consecutive_orders3.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders3 = df_consecutive_orders3.rename(columns={'ones':'consecutive_reorders'})

In [19]:
# Adding feature for consecutive orders
df_consecutive_orders4 = df_consecutive_orders.merge(df_users4, on=['user_id'])

df_consecutive_orders4 = (df_consecutive_orders4.merge(df_consecutive_orders4, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders4 = df_consecutive_orders4[df_consecutive_orders4['order_number_x'] - \
                                                  df_consecutive_orders4['order_number_y'] == 1]

df_consecutive_orders4['ones'] = 1

df_consecutive_orders4 = df_consecutive_orders4.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders4 = df_consecutive_orders4.rename(columns={'ones':'consecutive_reorders'})

In [20]:
# Adding feature for consecutive orders
df_consecutive_orders5 = df_consecutive_orders.merge(df_users5, on=['user_id'])

df_consecutive_orders5 = (df_consecutive_orders5.merge(df_consecutive_orders5, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders5 = df_consecutive_orders5[df_consecutive_orders5['order_number_x'] - \
                                                  df_consecutive_orders5['order_number_y'] == 1]

df_consecutive_orders5['ones'] = 1

df_consecutive_orders5 = df_consecutive_orders5.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders5 = df_consecutive_orders5.rename(columns={'ones':'consecutive_reorders'})

In [21]:
# Adding feature for consecutive orders
df_consecutive_orders6 = df_consecutive_orders.merge(df_users6, on=['user_id'])

df_consecutive_orders6 = (df_consecutive_orders6.merge(df_consecutive_orders6, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders6 = df_consecutive_orders6[df_consecutive_orders6['order_number_x'] - \
                                                  df_consecutive_orders6['order_number_y'] == 1]

df_consecutive_orders6['ones'] = 1

df_consecutive_orders6 = df_consecutive_orders6.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders6 = df_consecutive_orders6.rename(columns={'ones':'consecutive_reorders'})

In [22]:
# Adding feature for consecutive orders
df_consecutive_orders7 = df_consecutive_orders.merge(df_users7, on=['user_id'])

df_consecutive_orders7 = (df_consecutive_orders7.merge(df_consecutive_orders7, on=['user_id','product_id'])\
                           .sort_values(by=['user_id','order_number_x','order_number_y'], ascending=False))

df_consecutive_orders7 = df_consecutive_orders7[df_consecutive_orders7['order_number_x'] - \
                                                  df_consecutive_orders7['order_number_y'] == 1]

df_consecutive_orders7['ones'] = 1

df_consecutive_orders7 = df_consecutive_orders7.groupby(['product_id', 'user_id'])\
                                                        ['ones'].sum().reset_index()

    
df_consecutive_orders7 = df_consecutive_orders7.rename(columns={'ones':'consecutive_reorders'})

In [23]:
df_consecutive_orders_list = [df_consecutive_orders1, df_consecutive_orders2, df_consecutive_orders3, \
                             df_consecutive_orders4, df_consecutive_orders5, df_consecutive_orders6, \
                             df_consecutive_orders7]

In [24]:
df_consecutive_orders = pd.concat(df_consecutive_orders_list)

In [25]:
df_order_products_prior = df_order_products_prior.merge(df_consecutive_orders, on=['product_id','user_id'], how='left')

In [26]:
user_prod_features2 = ['user_product_consecutive_reorders','user_product_total_reorders']

df_user_prod_features2 = (df_order_products_prior.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict(
                                                     [('consecutive_reorders','sum'),
                                                      ('reordered','sum')])))

df_user_prod_features2.columns = ['product_id','user_id'] + user_prod_features2

In [27]:
df_X = df_X.merge(df_user_prod_features2,on=['user_id','product_id'])

df_X = df_X.fillna(-1)

df_X['user_product_percent_consecutive_reorders'] = df_X['user_product_consecutive_reorders'] / \
                                                    df_X['user_product_total_reorders']
    
df_X = df_X.fillna(-1)

In [28]:
# Save file out to csv
df_X.to_csv('instacart_df_X.csv', index=False)

In [7]:
# Adding feature for number of days since prior time product was ordered

df_days_cumulative = df_order_products_prior[['user_id','order_number','days_since_prior_order']]\
                            .sort_values(by=['user_id','order_number'], ascending=False)
    
df_days_cumulative = df_days_cumulative.drop_duplicates()

df_days_cumulative['cumulative_days'] = df_days_cumulative.groupby('user_id')['days_since_prior_order'].cumsum()

In [8]:
df_order_products_prior = df_order_products_prior.merge(df_days_cumulative, on=['user_id','order_number'], how='left')

In [9]:
user_prod_features2 = ['user_product_days_since_most_recent']

df_user_prod_features2 = (df_order_products_prior.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict(
                                                     [('cumulative_days','min')])))

df_user_prod_features2.columns = ['product_id','user_id'] + user_prod_features2

In [10]:
df_X = df_X.merge(df_user_prod_features2,on=['user_id','product_id'])

df_X = df_X.fillna(-1)

# Adding features for 3 most recent orders

In [36]:
df_X = pd.read_csv('instacart_df_X.csv')

In [29]:
df_max_order_user = (df_order_products_prior.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_number','max')])))

In [30]:
df_orders_user = (df_order_products_prior.groupby(['user_id','order_number'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('product_id','count')])))

df_orders_user = df_orders_user.drop(columns='product_id')

df_orders_user = df_orders_user.merge(df_max_order_user, on='user_id')

In [31]:
df_recent_orders = df_orders_user[df_orders_user['order_number_y'] - df_orders_user['order_number_x'] < 3]
df_recent_orders = df_recent_orders.drop(columns='order_number_y')
df_recent_orders = df_recent_orders.rename(columns={'order_number_x':'order_number'})

In [32]:
df_order_products_prior_recent = df_order_products_prior.merge(df_recent_orders, on=['user_id','order_number'])

In [37]:
user_features = ['user_total_orders_recent']

df_user_features = (df_order_products_prior_recent.groupby(['user_id'],as_index=False)
                                           .agg(OrderedDict(
                                                   [('order_id','nunique')])))

df_user_features.columns = ['user_id'] + user_features

df_X = df_X.merge(df_user_features, on='user_id', how='left')

df_X = df_X.fillna(-1)

In [38]:
user_prod_features = ['user_product_total_orders_recent','user_product_avg_add_to_cart_order_recent',\
                      'user_product_avg_order_dow_recent',\
                      'user_product_avg_order_hour_of_day_recent','user_product_avg_days_since_prior_order_recent']

df_user_prod_features = (df_order_products_prior_recent.groupby(['product_id','user_id'],as_index=False) \
                                                .agg(OrderedDict(
                                                     [('order_id','count'),
                                                      ('add_to_cart_order','mean'),
                                                      ('order_dow', 'mean'),
                                                      ('order_hour_of_day', 'mean'),
                                                      ('days_since_prior_order', 'mean')])))

df_user_prod_features.columns = ['product_id','user_id'] + user_prod_features 

df_X = df_X.merge(df_user_prod_features,on=['user_id','product_id'], how='left')

df_X['user_product_order_freq_recent'] = df_X['user_product_total_orders_recent'] / df_X['user_total_orders_recent'] 

df_X = df_X.fillna(-1)

In [39]:
# Save file out to csv
df_X.to_csv('instacart_df_X_final.csv', index=False)

In [11]:
df_X.to_csv('instacart_df_X_final2.csv', index=False)